# 📢📢📢 Projet NLP : La parole aux citoyens (Make.org - Le tourisme vert en Ille-et-Vilaine)📢📢📢
# Technique LDA
## Bibliothèques nécessaires

In [1]:
import pandas as pd
import plotly.express as px
from gensim.models import CoherenceModel, LdaMulticore
from gensim.corpora import Dictionary
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import ast

## Lecture des données

In [2]:
# Chargement des jeux de données
df = pd.read_parquet(
    "../data/processed/req_tourisme_responsable_embeddings.parquet",
    engine='pyarrow'
)

df.head(3)

,content,agree_count,disagree_count,neutral_count,agree_score,disagree_score,neutral_score,doNotCare,doNotUnderstand,doable,...,nb_exclamation,tokens,case,tokens_normalized,stop_words,tokens_final,tokens_lemmatized,embedding_prop,embedding_prop_x,embedding_prop_y
0,Il faut une surveillance des lieux touristique...,27,7,7,0.65,0.18,0.17,1,0,8,...,0,"['Il', 'faut', 'une', 'surveillance', 'des', '...","['Min', 'Min', 'Min', 'Min', 'Min', 'Min', 'Mi...","['une', 'surveillance', 'des', 'lieux', 'touri...","['il', 'une', 'des', 'avec', 'la', 'pour', 'ou']","[surveillance, lieux, amende, clef, degradatio...","['surveillance', 'lieux', 'touristique', 'amen...","[0.026475444436073303, 0.013547265902161598, 0...",60.746189,15.809195
1,Il faut donner des sacs poubelles et des cendr...,56,18,15,0.62,0.19,0.19,0,0,21,...,0,"['Il', 'faut', 'donner', 'des', 'sacs', 'poube...","['Min', 'Min', 'Min', 'Min', 'Min', 'Min', 'Mi...","['donner', 'des', 'sacs', 'poubelles', 'et', '...","['il', 'des', 'et', 'des', 'de', 'aux', 'des',...","[donner, sac, poubelle, cendrier, poch, vacanc...","['donner', 'sac', 'poubelle', 'cendrier', 'poc...","[0.034178830683231354, 0.05750420689582825, 0....",54.815662,12.159612
2,Il faut récompenser et mettre en avant les act...,35,7,14,0.62,0.13,0.25,0,5,12,...,0,"['Il', 'faut', 'récompenser', 'et', 'mettre', ...","['Min', 'Min', 'Min', 'Min', 'Min', 'Min', 'Mi...","['recompenser', 'et', 'mettre', 'en', 'avant',...","['il', 'et', 'en', 'les', 'des', 'du', 'et', '...","[recompenser, mettre, avant, acte, projet, eco...","['recompenser', 'mettre', 'avant', 'acte', 'pr...","[-0.011080465279519558, 0.053166456520557404, ...",-22.491800,47.501347


## LDA avec les differents nombre de clusters finalement retenus

In [3]:
df['tokens_final'] = df['tokens_final'].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else x
)

In [4]:
# Convertir les données de tokens en une liste de listes de tokens
processed_docs = df['tokens_final'].tolist()

# Créer un dictionnaire à partir des tokens
dictionary = Dictionary(processed_docs)

In [5]:
# Filtrer les tokens extrêmes
dictionary.filter_extremes(no_below=15, no_above=0.1, keep_n=1000)

In [6]:
# Convertir les documents en sacs de mots (Bag Of Words)
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

In [7]:
lda_model = LdaMulticore(bow_corpus, num_topics = 5, id2word = dictionary, passes = 50)

In [8]:
topics = []
for idx, topic in lda_model.print_topics(-1) :
    print("Topic: {} -> Words: {}".format(idx, topic))
    topics.append(topic)

Topic: 0 -> Words: 0.113*"local" + 0.072*"transport" + 0.056*"favoriser" + 0.040*"commun" + 0.032*"encourager" + 0.030*"moyen" + 0.027*"produire" + 0.023*"mode" + 0.021*"structure" + 0.021*"comme"
Topic: 1 -> Words: 0.060*"durable" + 0.053*"creer" + 0.050*"acteur" + 0.045*"pouvoir" + 0.043*"site" + 0.034*"etre" + 0.033*"region" + 0.030*"habitant" + 0.026*"responsable" + 0.023*"territoire"
Topic: 2 -> Words: 0.061*"limiter" + 0.044*"interdir" + 0.041*"lieux" + 0.039*"naturel" + 0.037*"site" + 0.035*"acce" + 0.027*"nombre" + 0.025*"arreter" + 0.025*"certain" + 0.024*"espace"
Topic: 3 -> Words: 0.060*"mettre" + 0.048*"camping" + 0.040*"place" + 0.037*"proposer" + 0.036*"responsable" + 0.033*"dechet" + 0.031*"vacance" + 0.028*"environnement" + 0.026*"nature" + 0.025*"lieux"
Topic: 4 -> Words: 0.095*"developper" + 0.070*"train" + 0.051*"velo" + 0.033*"rendre" + 0.030*"promouvoir" + 0.026*"cyclable" + 0.023*"offrir" + 0.022*"sejour" + 0.021*"piste" + 0.021*"voyage"


In [9]:
coherence_model_lda = CoherenceModel(model=lda_model, texts=processed_docs, dictionary=dictionary)
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)

Coherence Score:  0.3028342622495564


In [10]:
all_topic_model = []
for i in range(len(topics)):
  str = topics[i].split(' + ')
  topic_model = []
  for j in range(10):
    weight = str[j][0:5]
    word = str[j][7:len(str[j])-1]
    topic_model.append((weight, word))
  all_topic_model.append(topic_model)

In [11]:
all_topic_model

[[('0.113', 'local'),
  ('0.072', 'transport'),
  ('0.056', 'favoriser'),
  ('0.040', 'commun'),
  ('0.032', 'encourager'),
  ('0.030', 'moyen'),
  ('0.027', 'produire'),
  ('0.023', 'mode'),
  ('0.021', 'structure'),
  ('0.021', 'comme')],
 [('0.060', 'durable'),
  ('0.053', 'creer'),
  ('0.050', 'acteur'),
  ('0.045', 'pouvoir'),
  ('0.043', 'site'),
  ('0.034', 'etre'),
  ('0.033', 'region'),
  ('0.030', 'habitant'),
  ('0.026', 'responsable'),
  ('0.023', 'territoire')],
 [('0.061', 'limiter'),
  ('0.044', 'interdir'),
  ('0.041', 'lieux'),
  ('0.039', 'naturel'),
  ('0.037', 'site'),
  ('0.035', 'acce'),
  ('0.027', 'nombre'),
  ('0.025', 'arreter'),
  ('0.025', 'certain'),
  ('0.024', 'espace')],
 [('0.060', 'mettre'),
  ('0.048', 'camping'),
  ('0.040', 'place'),
  ('0.037', 'proposer'),
  ('0.036', 'responsable'),
  ('0.033', 'dechet'),
  ('0.031', 'vacance'),
  ('0.028', 'environnement'),
  ('0.026', 'nature'),
  ('0.025', 'lieux')],
 [('0.095', 'developper'),
  ('0.070', 'tra

In [12]:
pyLDAvis.enable_notebook()

In [13]:
# Préparer la visualisation
vis_data = gensimvis.prepare(lda_model, bow_corpus, dictionary)

In [14]:
# Enregistrer la visualisation en HTML
pyLDAvis.save_html(vis_data, '../outputs/lda.html')

In [15]:
# Afficher la visualisation dans le notebook
pyLDAvis.display(vis_data)

Avec la technique LDA, je trouve le decoupage intéressant.  
J'ai presque plus de facilité à mettre un nom sur un topic.  
On retrouve quand même des ressemblances avec ce que l'on a fait jusqu'à maintenant

In [16]:
# Créer une liste pour stocker les topics dominants de chaque document
dominant_topics = []

# Pour chaque document dans le corpus
for doc in bow_corpus:
    # Obtenir les topics du document avec leurs probabilités
    topics = lda_model.get_document_topics(doc, minimum_probability=0.0)
    # Si des topics sont disponibles pour le document
    if topics:
        # Trier les topics par probabilité et obtenir le topic le plus probable
        dominant_topic = max(topics, key=lambda x: x[1])[0]
        dominant_topics.append(dominant_topic)
    else:
        # Si aucun topic n'est disponible pour le document, attribuer -1
        dominant_topics.append(-1)

# check sur la derniere ligne
print(topics)
print(dominant_topic)  

[(0, 0.033901367), (1, 0.033945017), (2, 0.8647144), (3, 0.03344273), (4, 0.033996508)]
2


In [17]:
# Ajouter les topics dominants à votre DataFrame
df['dominant_topic'] = dominant_topics

In [18]:
df['dominant_topic'].value_counts()

dominant_topic
3    354
2    316
4    304
0    269
1    250
Name: count, dtype: int64

### Figure comparative avec les autres techniques

In [19]:
# Créer un graphique dynamique avec Plotly
fig = px.scatter(
    df[['content', 'embedding_prop_x', 'embedding_prop_y', 'dominant_topic']], 
    x='embedding_prop_x', 
    y='embedding_prop_y', 
    hover_name='content',
    color='dominant_topic', 
    color_continuous_scale='Viridis',  # Choisir une palette de couleurs
    range_color=[0, df['dominant_topic'].max()],  # Définir la plage de couleurs en fonction des étiquettes de cluster
    labels={'dominant_topic': 'Dominant Topic'},  # Renommer l'étiquette de la légende
    color_continuous_midpoint=int(df['dominant_topic'].max() / 2),  # Point médian de la palette de couleurs
    opacity=0.7
)
fig.update_traces(textposition='top center', showlegend=False)
fig.update_xaxes(visible=False)
fig.update_yaxes(visible=False)
fig.update_layout(
    title=dict(
        text='2D Sentence by LDA Dominant Topic',
        font=dict(
            size=25,
            color="black",
            family="Arial"
        ),
        xanchor="center",
        yanchor="top",
        x=0.5,
        y=0.95
    ),
    template='simple_white',
    hoverlabel_font_size=16,
    coloraxis_showscale=False,
)

fig.show()

On observe que l'utilisation de la technique LDA offre une représentation assez différente. Elle n'est pas directement comparable avec les méthodes classiques de NLP et d'embeddings. Cependant, cette technique peut être complémentaire et intéressante pour élargir les possibilités.